In [1]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout

datadir = os.path.join('.', 'dataset', 'brain_tumor_dataset')

def resize_and_padding(image, target_size):
    image = image.resize(target_size, Image.LANCZOS)
    return image

def convert_to_array(image):
    return np.array(image)

def load_and_preprocess(datadir, target_size=(224, 224)):
    images = []
    labels = []
    for label in ['yes', 'no']:
        folder_path = os.path.join(datadir, label)
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('RGB')
            image = resize_and_padding(image, target_size)
            image = convert_to_array(image)
            image = preprocess_input(image)  # Use VGG16 preprocessing
            images.append(image)
            labels.append(1 if label == 'yes' else 0)
    images = np.array(images)
    labels = np.array(labels)
    print(f"Loaded {len(images)} images with shape {images.shape}")
    return images, labels

def split_data(images, labels, test_size=0.2, random_state=42):
    return train_test_split(images, labels, test_size=test_size, random_state=random_state)

def create_data_generator(train_data, val_data, batch_size):
    train_images, train_labels = train_data
    val_images, val_labels = val_data
    
    train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    val_datagen = ImageDataGenerator()
    
    train_generator = train_datagen.flow(
        train_images, train_labels,
        batch_size=batch_size
    )
    
    val_generator = val_datagen.flow(
        val_images, val_labels,
        batch_size=batch_size
    )
    
    return train_generator, val_generator

def build_CNN_with_TL(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)  # Enable dropout
    x = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = (224, 224, 3)
model = build_CNN_with_TL(input_shape)
model.summary()

images, labels = load_and_preprocess(datadir)

train_images, val_images, train_labels, val_labels = split_data(images, labels)

batch_size = 64
train_data = (train_images, train_labels)
val_data = (val_images, val_labels)

train_generator, validation_generator = create_data_generator(
    train_data, val_data, batch_size
)

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_images) // batch_size,
    validation_data=validation_generator,
    validation_steps=len(val_images) // batch_size,
    epochs=10   
)



2024-08-01 17:25:44.845540: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 17:25:44.952040: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 17:25:44.953492: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-01 17:25:46.640301: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

2024-08-01 17:25:57.520630: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 128450560 exceeds 10% of free system memory.
2024-08-01 17:25:57.756436: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 128450560 exceeds 10% of free system memory.


1/3 [=========>....................] - ETA: 14s - loss: 4.1048 - accuracy: 0.5000

2024-08-01 17:26:02.766172: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 822083584 exceeds 10% of free system memory.
2024-08-01 17:26:04.548086: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 822083584 exceeds 10% of free system memory.
2024-08-01 17:26:09.404668: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


3/3 [==============================] - 73s 33s/step - loss: 15.2555 - accuracy: 0.5290
Epoch 2/10
3/3 [==============================] - 92s 28s/step - loss: 13.4182 - accuracy: 0.6771
Epoch 3/10
3/3 [==============================] - 63s 16s/step - loss: 7.3036 - accuracy: 0.6957
Epoch 4/10
3/3 [==============================] - 62s 28s/step - loss: 4.1703 - accuracy: 0.7826
Epoch 5/10
3/3 [==============================] - 92s 30s/step - loss: 5.1187 - accuracy: 0.7812
Epoch 6/10
3/3 [==============================] - 60s 16s/step - loss: 3.5311 - accuracy: 0.8188
Epoch 7/10
3/3 [==============================] - 80s 26s/step - loss: 2.6113 - accuracy: 0.8073
Epoch 8/10
3/3 [==============================] - 60s 16s/step - loss: 3.2610 - accuracy: 0.7609
Epoch 9/10
3/3 [==============================] - 80s 26s/step - loss: 2.3210 - accuracy: 0.8281
Epoch 10/10
3/3 [==============================] - 57s 15s/step - loss: 3.6703 - accuracy: 0.7971


In [2]:
model.save('TF_brain_tumor_detection_model.h5')

def predict(image):
    image = resize_and_padding(image, (224, 224))  # Ensure the target size matches your model's input
    image = convert_to_array(image)
    image = preprocess_input(image)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    prediction = model.predict(image)
    return 'Tumor' if prediction[0] > 0.5 else 'No Tumor'

model = tf.keras.models.load_model('TF_brain_tumor_detection_model.h5')


/home/sardar/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
